In [2]:
#Name: CS 458 Task 1 Testing File Predictions and Model Analysis
#Authors: Liliana Pacheco, Chantelle Suarez, Yan Tarpley
#Name: CS 458 Task 3 Person Type
#Authors: Liliana Pacheco, Chantelle Suarez, Yan Tarpley
#Date: December 9, 2019
#Description: This code takes in the UsedAutoRELEVATEFfirst10000-noLatLong.csv excel sheet and then processes the data
#by doing the follwoing thins:
#   Removing empty columns and rows
#   converting all non numeric data
#   filling in missing values with the average for the columns
#   removing columns with data that has more than 60% missing values
#This code then goes on to analyse the data set for the best model and then making predictions for 
#the data set. The data set is splitt to 80% percent training to 20% testing and predictions or done for a target of
#Person Type. Vehicle Type was analysed and evaluated on a different Code for readibility.
#NOTE: The model analysis portion of this program was created using Jason Brownlee's article 
#"How To Compare Machine Learning Algorithms in Python with scikit-learn" as a referrence
#Link to the article: https://machinelearningmastery.com/compare-machine-learning-algorithms-python-scikit-learn/


# Load libraries
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

#----------------------------------------------------------------------------------------------#
#                                 May Change File Paths Here                                   #

personDF = pd.read_excel(r"C:\Users\lilia_fdv6j62\Documents\autoData.xlsx", header=0)

#----------------------------------------------------------------------------------------------#


#---------------------------------------------------------------------------------------
#*    This Function was written using the following referrence
#*    Title: Handling Non-Numerical Data for Machine Learning
#*    Author: Harrison@pythonprogramming.net
#*    Date: December 9, 2019
#*    Availability: https://pythonprogramming.net/working-with-non-numerical-data-machine-learning-tutorial/
#*
#---------------------------------------------------------------------------------------
def convertLabels(df):
    columns = df.columns.values
    for column in columns:
        textValues = {}
        def convert(val):
            return textValues[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            contents = df[column].values.tolist()
            uniqueVal = set(contents)
            x = 0
            for unique in uniqueVal:
                if unique not in textValues:
                    textValues[unique] = x
                    x+=1
            df[column] = list(map(convert, df[column])) 
                
    return df
#-----------------------------------------------------------------------------------------

#remove all empty columns in the dataframe
personDF = personDF.dropna(axis = 1, how = 'all')

#Move Target column "Person Type" to the end/right most column
personDF = personDF[personDF['Person Type'].notnull()]
personDFTarget = personDF.pop('Person Type')
personDF['Person Type'] = personDFTarget

#remove columns with more than 60% of data missing
personDF = personDF.loc[:, personDF.isin([' ','NULL',0]).mean() < .4]
#fill in any missing values with the average
personDF.fillna(personDF.mean(), inplace = True)
#Make All Data Numeric
personDf = convertLabels(personDF)

#Split Training Data For Model Analysis
finalData = personDF.values
xVal = finalData[:,0:len(personDF.columns) - 1]
yVal= finalData[:,len(personDF.columns) - 1]
xTrain, xTest, yTrain, yTest = train_test_split(xVal, yVal, test_size=0.20, random_state=1)
#Select Models For Testing
models = []
models.append(('LR ', LogisticRegression(solver='liblinear',multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNC', KNeighborsClassifier()))
models.append(('DTC', DecisionTreeClassifier()))
models.append(('GNB', GaussianNB()))

#Evaluate Each Model
print("\n\n#-----------------------------------------------------------------------------------------------------#")
print("#--------------Model Analysis and Testing With UsedAutoRELEVANTfirst10000-noLatLong.csv---------------#")
print("#---------------------------------For Customer Type / Person Type-------------------------------------#")
print("     Accuracy AVG Diviation")
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=0)
    cv_results = model_selection.cross_val_score(model, xTrain, yTrain, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    
#Select The Best Model For Testing
print("The best model results in a Decision Tree Model.")
testModel =  DecisionTreeClassifier()
testModel.fit(xTrain, yTrain)

#Make Sample Predictions
testPredictions = testModel.predict(xTest)
print("\nNumeric to Person Type Key: ")
print (personDF.iloc[0]['Person Type'], " >> P")
print (personDF.iloc[1]['Person Type'], " >> E")
print (personDF.iloc[7]['Person Type'], " >> Y")
print (personDF.iloc[11]['Person Type'], " >> O")

print("\nSample Predictions: ")
print(testPredictions)
print("\nSample Actual:")
print(yTest)

# Evaluate Sample predictions
print("\nSample Accuracy:")
print(accuracy_score(yTest, testPredictions))

print("\nSample Report")
print(classification_report(yTest, testPredictions))



#-----------------------------------------------------------------------------------------------------#
#--------------Model Analysis and Testing With UsedAutoRELEVANTfirst10000-noLatLong.csv---------------#
#---------------------------------For Customer Type / Person Type-------------------------------------#
     Accuracy AVG Diviation
LR : 0.836640 (0.015516)
LDA: 0.925948 (0.011077)
KNC: 0.821152 (0.015990)
DTC: 0.988143 (0.006253)
GNB: 0.818007 (0.015812)
The best model results in a Decision Tree Model.

Numeric to Person Type Key: 
2.0  >> P
1.0  >> E
0.0  >> Y
3.0  >> O

Sample Predictions: 
[2. 2. 2. ... 2. 2. 3.]

Sample Actual:
[2. 2. 2. ... 2. 2. 3.]

Sample Accuracy:
0.9738878143133463

Sample Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        17
         1.0       0.64      0.59      0.62        27
         2.0       1.00      0.99      1.00       853
         3.0       0.89      0.92      0.90       137

    